# feature extraction and embeddings

In [60]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from gensim.scripts.glove2word2vec import glove2word2vec
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.model_selection import train_test_split
from gensim.models.keyedvectors import KeyedVectors
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from gensim.models import Word2Vec, FastText
from sklearn.pipeline import Pipeline
from collections import defaultdict
from sklearn.svm import LinearSVC
import gensim.downloader as api
import pandas as pd
import numpy as np
import random
import time

## A. preparation de donnees

In [2]:
df = pd.read_csv('./spooky.csv')

In [3]:
df.head()

,id,text,author
0,id26305,this proces however afforded me no means of as...,EAP
1,id17569,it never once occurred to me that the fumbling...,HPL
2,id11008,in his left hand was a gold snuff box from whi...,EAP
3,id27763,how lovely is spring as we looked from windsor...,MWS
4,id12958,finding nothing else not even gold the superin...,HPL


In [4]:
df['text'].isnull().sum()

0

## B. encodage de la variable a predire (facultatif)

In [5]:
label_encoder = LabelEncoder()
df['author_encoded'] = label_encoder.fit_transform(df['author'])

In [6]:
df.head()

,id,text,author,author_encoded
0,id26305,this proces however afforded me no means of as...,EAP,0
1,id17569,it never once occurred to me that the fumbling...,HPL,1
2,id11008,in his left hand was a gold snuff box from whi...,EAP,0
3,id27763,how lovely is spring as we looked from windsor...,MWS,2
4,id12958,finding nothing else not even gold the superin...,HPL,1










## C. construction des bases d’entraînement et de test

 tarining & test dataset

In [7]:
x_train, x_test, y_train, y_test = train_test_split(df['text'], df['author_encoded'], test_size=0.3, random_state=0, stratify = df['author_encoded'])

In [8]:
x_train

8446     upon these two words therefore i have mainly b...
15332    i had the good fortune to recollect that in th...
4486     each one of a gang so placed is not so much gr...
19359         if he were vanquished i should be a free man
6440     among a multitude of opinions upon this delica...
                               ...                        
7389     at these times he would shew a sardonic humour...
15422    made sign talk as soon as they got over bein s...
2581      there was no discoloration in the cellular tisue
4597                        his forehead was broad and low
1951     as i perused this i felt myself growing gradua...
Name: text, Length: 13705, dtype: object

## G. vectorisation (embeddings de mots)

glove

In [10]:
cleaned_df = [sentence.strip().strip('"') for sentence in x_train]

with open("corpus.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(cleaned_df))

print("save .txt file")


save .txt file


In [24]:
cd GloVe; make

[Errno 2] No such file or directory: 'GloVe; make'
/home/mina/projects/NLP_labs/Tp2/GloVe


In [25]:
pwd

'/home/mina/projects/NLP_labs/Tp2/GloVe'

build vocabulary from corpus.txt

In [32]:
import subprocess

In [35]:
subprocess.run('pwd')

/home/mina/projects/NLP_labs/Tp2/GloVe


CompletedProcess(args='pwd', returncode=0)

In [49]:
with open("../corpus.txt", "r") as corpus_file, open("../vocab.txt", "w") as vocab_file:
    subprocess.run(["build/vocab_count", "-min-count", "5", "-verbose", "2"],
                   stdin=corpus_file, stdout=vocab_file, check=True)

BUILDING VOCABULARY
Processed 0 tokens.100000 tokens.200000 tokens.300000 tokens.Processed 367653 tokens.
Counted 23413 unique words.
Truncating vocabulary at min count 5.
Using vocabulary of size 6897.



generate co-occurrence matrix

In [52]:
ls build

common.o  cooccur.o  glove.o   shuffle.o     vocab_count.o
cooccur*  glove*     shuffle*  vocab_count*


In [ ]:
subprocess.run('build/cooccur -window-size 15 -vocab-file ../vocab.txt -verbose 2 < ../corpus.txt > ../cooccurrence.bin', shell=True)

COUNTING COOCCURRENCES
window size: 15
context: symmetric
max product: 10485784
overflow length: 28521267
Reading vocab from file "../vocab.txt"...loaded 6897 words.
Building lookup table...table contains 26336491 elements.
Processing token: 0100000200000300000Processed 367653 tokens.
Writing cooccurrences to disk.......2 files in total.
Merging cooccurrence files: processed 0 lines.100000 lines.200000 lines.300000 lines.400000 lines.500000 lines.600000 lines.700000 lines.800000 lines.900000 lines.1000000 lines.1100000 lines.1200000 lines.1300000 lines.1400000 lines.1500000 lines.1600000 lines.1700000 lines.Merging cooccurrence files: processed 1762742 lines.



CompletedProcess(args='build/cooccur -window-size 15 -vocab-file ../vocab.txt -verbose 2 < ../corpus.txt > ../cooccurrence.bin', returncode=0)

shuffle co-occurrence matrix

In [58]:
subprocess.run('build/shuffle -verbose 2 < ../cooccurrence.bin > ../cooccurrence.shuf.bin', shell=True)

Using random seed 1740615843
SHUFFLING COOCCURRENCES
array size: 127506841
Shuffling by chunks: processed 0 lines.processed 1762742 lines.
Wrote 1 temporary file(s).
Merging temp files: processed 0 lines.1762742 lines.Merging temp files: processed 1762742 lines.



CompletedProcess(args='build/shuffle -verbose 2 < ../cooccurrence.bin > ../cooccurrence.shuf.bin', returncode=0)

train the model

In [59]:
subprocess.run('build/glove  -save-file ../vectors -threads 4 -input-file ../cooccurrence.shuf.bin -x-max 10 -iter 50 -vector-size 100 -binary 2 -vocab-file ../vocab.txt -verbose 2', shell=True)

TRAINING MODEL
Read 1762742 lines.
Initializing parameters...Using random seed 1740615845
done.
vector size: 100
vocab size: 6897
x_max: 10.000000
alpha: 0.750000
02/27/25 - 01:24.10AM, iter: 001, cost: 0.062950
02/27/25 - 01:24.13AM, iter: 002, cost: 0.046853
02/27/25 - 01:24.17AM, iter: 003, cost: 0.042858
02/27/25 - 01:24.19AM, iter: 004, cost: 0.040662
02/27/25 - 01:24.22AM, iter: 005, cost: 0.038793
02/27/25 - 01:24.25AM, iter: 006, cost: 0.036499
02/27/25 - 01:24.29AM, iter: 007, cost: 0.034098
02/27/25 - 01:24.32AM, iter: 008, cost: 0.031935
02/27/25 - 01:24.36AM, iter: 009, cost: 0.029918
02/27/25 - 01:24.39AM, iter: 010, cost: 0.028069
02/27/25 - 01:24.43AM, iter: 011, cost: 0.026435
02/27/25 - 01:24.46AM, iter: 012, cost: 0.024989
02/27/25 - 01:24.49AM, iter: 013, cost: 0.023702
02/27/25 - 01:24.53AM, iter: 014, cost: 0.022517
02/27/25 - 01:24.56AM, iter: 015, cost: 0.021503
02/27/25 - 01:24.59AM, iter: 016, cost: 0.020601
02/27/25 - 01:25.02AM, iter: 017, cost: 0.019772
02/2

CompletedProcess(args='build/glove  -save-file ../vectors -threads 4 -input-file ../cooccurrence.shuf.bin -x-max 10 -iter 50 -vector-size 100 -binary 2 -vocab-file ../vocab.txt -verbose 2', returncode=0)

use the trained model

In [68]:
cd ..

/home/mina/projects/NLP_labs/Tp2


In [69]:
ls

 cooccurrence.bin        GloVe/            skip-gram.ipynb   vectors.bin
 cooccurrence.shuf.bin   glove.ipynb       spooky.csv        vectors.txt
 corpus.txt              rapport_tp2.odt  'TP02 2025.pdf'    vocab.txt
 fastText.ipynb          rapport_tp2.pdf   tp2.ipynb


In [70]:
glove_file = "vectors.txt"
word2vec_file = "vectors.word2vec"

glove2word2vec(glove_file, word2vec_file)

/tmp/ipykernel_14379/3950756513.py:4: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_file, word2vec_file)


(6898, 100)

save the model

In [74]:
model = KeyedVectors.load_word2vec_format(word2vec_file, binary=False)

model.save("glove_model.kv")

apply glove

In [75]:
def text_to_glove_vectors(text_data, model, vector_size=100):
    vectors = []
    for sentence in text_data:
        words = sentence.split()
        word_vectors = [model[word] for word in words if word in model]
        if word_vectors:
            sentence_vector = np.mean(word_vectors, axis=0)
        else:
            sentence_vector = np.zeros(vector_size) 
        vectors.append(sentence_vector)
    return np.array(vectors)

In [76]:
x_train_glove = text_to_glove_vectors(x_train, model)
x_test_glove = text_to_glove_vectors(x_test, model)

the MPLClassifier

In [73]:
def train_and_evaluate(X_train, X_test, y_train, y_test):
    mlp = MLPClassifier(hidden_layer_sizes=(20,), max_iter=20, random_state=0)
    mlp.fit(X_train, y_train)
    
    y_train_pred = mlp.predict(X_train)
    y_test_pred = mlp.predict(X_test)
    
    print("train classification report:")
    print(classification_report(y_train, y_train_pred))
    print("test classification report:")
    print(classification_report(y_test, y_test_pred))
    
    start_time = time.time()
    mlp.predict(X_test)
    print(f"prediction time: {time.time() - start_time:.5f} seconds")

In [77]:
train_and_evaluate(x_train_glove, x_test_glove, y_train, y_test)

train classification report:
              precision    recall  f1-score   support

           0       0.70      0.69      0.70      5530
           1       0.64      0.65      0.64      3944
           2       0.67      0.67      0.67      4231

    accuracy                           0.67     13705
   macro avg       0.67      0.67      0.67     13705
weighted avg       0.67      0.67      0.67     13705

test classification report:
              precision    recall  f1-score   support

           0       0.68      0.66      0.67      2370
           1       0.63      0.63      0.63      1691
           2       0.64      0.67      0.66      1813

    accuracy                           0.65      5874
   macro avg       0.65      0.65      0.65      5874
weighted avg       0.65      0.65      0.65      5874

prediction time: 0.00474 seconds


/home/mina/anaconda3/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(
